# Create Monthly Task Reports in Smartsheet


In [ ]:
%load_ext nb_black

import os
import json
import logging
from datetime import datetime, timedelta
from typing import Dict, List

import pandas as pd
import numpy as np
import prefect
from box import Box

import smartsheet

# uses the pretty okay SDK here: https://github.com/ProdPerfect/monday
from monday import MondayClient
from mondaydotcom_utils.formatted_value import FormattedValue, get_col_defs
from mondaydotcom_utils.time_block import TimeBlock
from mondaydotcom_utils.utilities import (
    breakout_record,
    get_items_by_board,
    validate_task_record,
)
from prefect import Flow, Parameter, task, unmapped
from prefect.executors import LocalDaskExecutor, LocalExecutor

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [ ]:
TASKS_BOARD_ID = "1883170887"
AGREEMENTS_BOARD_ID = "1882423671"
PROJECTS_BOARD_ID = "1882404316"
ACCOUNTS_BOARD_ID = "1882424009"

PROJECT_TASK_TIME_BOARD_ID = "2398200403"

# don't set this here for development work... use the secrets-<environment>.yaml files instead.
MONDAY_KEY = ""
SMARTSHEET_KEY = ""
environment = "dev"

# change these or set as papermill parameters to report on year and month
year_for_report = 2022
month_for_report = 3

In [ ]:
if not MONDAY_KEY:
    # key hasn't been passed as a papermill parameter... get it from a file?
    secrets = Box.from_yaml(filename=f"secrets-{environment}.yaml")
    MONDAY_KEY = secrets.apps.monday.API_KEY

if not SMARTSHEET_KEY:
    # key hasn't been passed as a papermill parameter... get it from a file?
    secrets = Box.from_yaml(filename=f"secrets-{environment}.yaml")
    SMARTSHEET_KEY = secrets.apps.smartsheet.API_KEY

In [ ]:
# connect monday client
conn = MondayClient(MONDAY_KEY)

In [ ]:
# connect smartsheet client
ss_client = smartsheet.Smartsheet(SMARTSHEET_KEY)
ss_client.errors_as_exceptions(True)

In [ ]:
users = conn.users.fetch_users()["data"]["users"]
users_df = pd.DataFrame(users).set_index("id")
users_df.head()

In [ ]:
def breakdown_status(x):
    # use this to break down the status columns
    # TODO move this to mondaydotcom-utils in the formatters

    my_list = []
    json1 = json.loads(x)

    if json1.get("text"):
        my_list.append(json1["text"])
    if json1.get("changed_at"):
        my_list.append(json1["changed_at"])

    return ";".join(my_list)

In [ ]:
accounts_df = get_items_by_board(conn, ACCOUNTS_BOARD_ID)

accounts_df.rename(
    columns={"monday_id": "account_id", "Title": "Client Name"},
    inplace=True,
)

accounts_df.drop(
    columns=[
        "Contacts",
        "Item ID",
        "Subitems",
        "Notes",
        "Customer Projects",
        "Agreements",
        "Type",
    ],
    inplace=True, errors="ignore"
)

# convert the yes-no to True-False
accounts_df["No Bill"] = accounts_df["No Bill"].apply(
    lambda x: bool(json.loads(x)["checked"]) if x else False
)

accounts_df

In [ ]:
projects_df = get_items_by_board(conn, PROJECTS_BOARD_ID)

projects_df.rename(
    columns={
        "monday_id": "project_id",
    },
    inplace=True,
)

projects_df.drop(
    columns=[
        "Repo Description (mirror)",
        "Project Tasks",
        "Subitems",
        "Etimated Time (Hours) (mirror)",
        "Total Task Time (Hours) (mirror)",
        "Project Contacts",
        "SET Resource",
        "Timeline",
        "Customer Source",
        "Tasks Status (mirror)",
        "Dependency",
        "Date Added",
        "Time Balance (Hours) (formula)",
        "Agreement NTE Hours (mirror)",
        "Timeline Days",
        "Item ID",
        "Project Health",
        "Notes",
        "Agreements",
    ],
    inplace=True, errors="ignore"
)

projects_df["Project Lifecycle"] = projects_df["Project Lifecycle"].apply(
    breakdown_status
)

projects_df = projects_df.explode(["Account"], ignore_index=True)
projects_df

In [ ]:
# add the account to the projects
projects_df = pd.merge(
    projects_df, accounts_df, how="left", left_on="Account", right_on="account_id"
).drop(columns=["account_id"])
projects_df

In [ ]:
def breakout_time_sessions(row):
    """
    Break down the Monday.com time structure into something simpler for us.

    This is used with a DataFrame.apply()
    """

    mct = TimeBlock()
    mct.parse(row["Actual Time"])
    return mct.total_duration_hours, mct.time_records

In [ ]:
# only getting done tasks
tasks_df = get_items_by_board(conn, TASKS_BOARD_ID, "status", "Done")

tasks_df.rename(
    columns={
        "monday_id": "task_id",
    },
    inplace=True,
)

tasks_df.drop(
    columns=[
        "Subtasks",
        "Timeline Hours (Estimated) (formula)",
        "Total Actual Hours (formula)",
        "Customer Repos",
        "Billing Agreement",
        "Project Lifecycle (mirror)",
    ],
    inplace=True, errors="ignore"
)

# break the time sessions out
tasks_df[["Total Duration Hours", "Time Sessions"]] = tasks_df.apply(
    breakout_time_sessions, axis=1, result_type="expand"
)

# Only include Ready tasks
tasks_df = tasks_df.loc[
    tasks_df["Integration Message"].str.startswith("Ready", na=False)
]

# projects should be limited to just one, so this will bring it out of the list
tasks_df = tasks_df.explode(["Customer Project"], ignore_index=True)
tasks_df.head()

Break down the Monday.com session items into individual "journal tasks".

In [ ]:
journal_items = []
records = tasks_df.to_dict(orient="records")
for record in records:
    new_list = breakout_record(record, users_df)

    # go through those N records, one by one
    for item in new_list:
        journal_items.append(item)

journal_task_df = pd.DataFrame(journal_items)

# break out the actual task status also; we've already used the changed_at
# field to help break the records out... so this can be simplified for info
journal_task_df["task_status"] = journal_task_df["Status"].apply(
    lambda x: json.loads(x)["text"] if x else None
)

# convert to a dataframe date... a bit crude for filtering
journal_task_df["task_end_date"] = pd.to_datetime(journal_task_df["task_end_date"])
journal_task_df["task_end_year"] = pd.DatetimeIndex(
    journal_task_df["task_end_date"]
).year
journal_task_df["task_end_month"] = pd.DatetimeIndex(
    journal_task_df["task_end_date"]
).month

journal_task_df.head()

In [ ]:
# only interested in this month's tasks
mask = (journal_task_df["task_end_year"] == year_for_report) & (
    journal_task_df["task_end_month"] == month_for_report
)
journal_task_df = journal_task_df.loc[mask]

# Some of this helped build the record, some is just mirror or lookup gak.
journal_task_df.drop(
    columns=[
        "Actual Hours",
        "Actual Time",
        "Customer Repos",
        "Date Added",
        "Date Completed",
        "Dependencies",
        "Integration Message",
        "Subtasks",
        "Timeline",
        "Total Actual Hours (formula)",
        "Total Duration Hours",
        "Time Sessions",
        "Owner",
        "Status",
        "Timeline Hours (Estimated) (formula)",
        "Project Lifecycle (mirror)",
        "Billing Agreement",
        "Timeline Days",
        "task_status",
    ],
    inplace=True,
    errors="ignore",
)

journal_task_df

Finally merge the tasks and projects together for a final task list.

In [ ]:
def month_end_date(year, month):
    """Calculate the month end date given a year and month."""
    month += 1
    if month == 13:
        month = 1
        year += 1

    tempdate = datetime.strptime(f"{year}-{month}-1", "%Y-%m-%d")
    return (tempdate - timedelta(days=1)).strftime("%Y-%m-%d")

In [ ]:
def month_end_me(row):
    return month_end_date(row["task_end_year"], row["task_end_month"])

In [ ]:
df = pd.merge(
    journal_task_df,
    projects_df,
    how="left",
    left_on="Customer Project",
    right_on="project_id",
)

# We only the wants those we bill for
df = df.loc[~df["No Bill"]]

df["Month Ending Date"] = df.apply(month_end_me, axis=1)

df.rename(
    columns={
        "monday_id_x": "monday_id",
        "monday_id_y": "project_id",
        "Title_x": "Title",
        "Title_y": "Project Title",
        "Notes_x": "Notes",
        "Notes_y": "Project Notes",
        "hours": "Hours",
        "owner": "Resource",
    },
    inplace=True,
)

df.drop(
    columns=[
        "project_id",
        "monday_id",
        "task_end_year",
        "task_end_month",
        "Account",
        "No Bill",
    ],
    inplace=True,
    errors="ignore",
)

df

In [ ]:
# create a group by report and post to SE Project/Grant Time smartsheet
report_df = (
    df.groupby(["Client Name", "Project Title", "Resource"])
    .agg(
        {
            "Hours": "sum",
            "Month Ending Date": "first",
            "Grant Number": "first",
        }
    )
    .reset_index()
)
report_df

In [ ]:
def filter_tasks(df, client, project, resource, month_end_date):
    """Get the journal tasks based on the details we'll send to smartsheet."""
    return df[
        (
            (df["Client Name"] == client)
            & (df["Project Title"] == project)
            & (df["Resource"] == resource)
            & (df["Month Ending Date"] == month_end_date)
        )
    ]

Now, Smartsheet's turn?

In [ ]:
# sheet_name = "SE Project Posted Time"

# search_results = ss_client.Search.search(sheet_name).results

# # helpful: https://stackoverflow.com/questions/52065527/python-best-way-to-get-smartsheet-sheet-by-name
# time_sheet_id = next(
#     result.object_id for result in search_results if result.object_type == "sheet"
# )
# time_sheet = ss_client.Sheets.get_sheet(time_sheet_id)
# time_sheet_id

In [ ]:
# smartsheet can have duplicate sheet names,
# so best not to rely on them
time_sheet_id = 3567675495475076

In [ ]:
# break down the cell IDs into a quick lookup box
cell_ids = {}
for column in time_sheet.columns:
    my_column = column.to_dict()
    cell_ids[my_column["title"]] = my_column["id"]
cell_ids

Add the records to Smartsheet

In [ ]:
rows = []
for k, v in report_df.to_dict("index").items():

    row = ss_client.models.row.Row()

    row.cells.append(
        {"column_id": cell_ids["Account/Client"], "value": v["Client Name"]}
    )
    if v.get("Grant Number"):
        row.cells.append(
            {"column_id": cell_ids["Grant Proposal #"], "value": v["Grant Number"]}
        )
    row.cells.append(
        {"column_id": cell_ids["Project Title"], "value": v["Project Title"]}
    )
    row.cells.append(
        {"column_id": cell_ids["Month-end Date"], "value": v["Month Ending Date"]}
    )
    row.cells.append({"column_id": cell_ids["Completed Hours"], "value": v["Hours"]})
    row.cells.append({"column_id": cell_ids["Resource"], "value": v["Resource"]})

    row.to_bottom = True
    rows.append(row)


result = ss_client.Sheets.add_rows(time_sheet_id, rows)

In [ ]:
# get the row ids and create a Series
my_list = []
for row in result.to_dict()["data"]:
    my_list.append(row["id"])


row_series = pd.Series(my_list, name="row_id", dtype=np.int64)
row_series

report_df = pd.concat([report_df, row_series], axis=1)
report_df

In [ ]:
# create a dictionary to make attaching the files easier
new_dict = {}
for k, v in report_df.to_dict("index").items():
    row_id = v["row_id"]

    # add the filtered tasks to a list
    new_dict[row_id] = filter_tasks(
        df,
        v["Client Name"],
        v["Project Title"],
        v["Resource"],
        v["Month Ending Date"],
    )

In [ ]:
# attach file to each record
if not os.path.exists("_cache"):
    os.mkdir("_cache")

for k, v in new_dict.items():
    filename = os.path.join("_cache", f"{k}.csv")

    # save file
    v.to_csv(filename, index=False)

    with open(filename, "r") as f:
        ss_client.Attachments.attach_file_to_row(time_sheet_id, k, f)

In [ ]:
for k, v in df.to_dict("index").items():
    result = conn.items.change_item_value(
        TASKS_BOARD_ID, v["task_id"], "text01", f"Posted - {datetime.now()}"
    )